In [ ]:
import os
from jsonschema_rs import validator_for
import json

from jsonschema import validate


In [ ]:
from pbcore.models.models import PBCore
from json import dumps
SCHEMA='schemas/pbcore.schema.json'

with open(SCHEMA, 'w') as f:
    f.write(dumps(PBCore.model_json_schema(), indent=2))

In [ ]:
DIRECTORY = 'pbcore-json'

with open(SCHEMA, 'r') as schema_file:
    schema = json.load(schema_file)

pb = validator_for(schema)

# with open(os.path.join(DIRECTORY, 'cpb-aacip_15-29p2p5g8.json'), 'r') as f:
with open(os.path.join(DIRECTORY, 'cpb-aacip-15-nc5s756s32.json'), 'r') as f:
    json_data = json.load(f)

# json_data

# validate_json(json_data)
# validate(json_data, schema)
pb.validate(json_data)

In [ ]:
errors = []
success = []
for path, subdirs, files in os.walk(DIRECTORY):
    for name in files:
        if name.startswith('cpb-aacip') and name.endswith('.json'):
            with open(os.path.join(path, name), 'r') as f:
                json_data = json.load(f)
                try:
                    # validate(json_data, schema) # jsonschema
                    pb.validate(json_data) # jsonschema_rs
                    # PBCore.model_/validate(json_data) # pydantic
                    # print(f"{name} is valid.")
                    success.append(name)
                except Exception as e:
                    print(f"{name} is invalid: {e.message}")
                    errors.append((name, e.message))


In [ ]:
len(success), len(errors), f'{(len(success)/(len(success) + len(errors))):.2%}'

In [ ]:
e = errors[0]
e

In [ ]:
mvp={
        "pbcoreDescriptionDocument": {
            "xsi:schemaLocation": "http://www.example.com/schema",
            "pbcoreIdentifier": [{"text": "123", "source": "example"}],
            "pbcoreTitle": [{"text": "Title"}],
            "pbcoreDescription": [{"text": ""}],
        },
    }
# del mvp['pbcoreDescriptionDocument']['pbcoreIdentifier'][0]['source']

# pb.validate(mvp)

In [ ]:
PBCore.model_validate(mvp)

In [ ]:
p = PBCore(**mvp)


In [ ]:
p.pbcoreDescriptionDocument.pbcoreIdentifier
